In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import autogluon
from autogluon.tabular import TabularDataset,TabularPredictor
from sklearn.model_selection import train_test_split
import utils
import geopandas as gpd

In [6]:
# pandasの表示設定
pd.set_option('display.max_columns', None)  # すべての列を表示
#pd.set_option('display.max_rows', None)     # すべての行を表示（オプション）
pd.set_option('display.width', None)        # 行の折り返しを防ぐ
pd.set_option('display.max_colwidth', None) # 列の内容を省略しない

In [ ]:
drop_cols=[
    "nl","el",'building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing'
    ]
land_price_cols=[
    "L02_001",#用途区分
    "L02_006",#調査価格
    "L02_024",#地積
    "L02_029",#水道
    "L02_037",#道路区分
    "L02_045",#最寄り駅までの道路距離
    "L02_055",
    "L02_056",
    "L02_057",
    "L02_058",
    "L02_059",
    "L02_060",
    "L02_061",
    "L02_062",
    "L02_063",
    "L02_064",
    "L02_065",
    "L02_066",
    "L02_067",
    "L02_068",
    "L02_069",
    "L02_070",
    "L02_071",
    "L02_072",
    "L02_073",
    "L02_074",
    "L02_075",
    "L02_076",
    "L02_077",
    "L02_078",
    "L02_079",
    "L02_080",
    "L02_081",
    "L02_082",
    "L02_083",
    "L02_084",
    "L02_085",
    "L02_086",
    "L02_087",
    "L02_088",
    "L02_089",
    "L02_090",
    "L02_091",
    "L02_092",
    "L02_093",
    "L02_094",
    "geometry"
    ]

In [49]:
gdf_json = gpd.read_file(os.path.join('..', 'data', 'train', 'L02-23.geojson'))#[land_price_cols]

In [50]:
train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))

C:\Users\km20186\AppData\Local\Temp\ipykernel_32892\3353662060.py:1: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
C:\Users\km20186\AppData\Local\Temp\ipykernel_32892\3353662060.py:2: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))
C:\Users\km20186\AppData\Local\Temp\ipykernel_32892\3353662060.py:2: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [51]:
train_gdf=utils.convert_to_geodataframe(train_df)
train_gdf=utils.nearest_merge(train_gdf,gdf_json,threshold=1500)
#display(train_gdf.head())


In [45]:
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))
test_gdf=utils.convert_to_geodataframe(test_df)
test_gdf=utils.nearest_merge(test_gdf,gdf_json,threshold=1500)
test_gdf["land_price_x"]=test_gdf.geometry.x
test_gdf["land_price_y"]=test_gdf.geometry.y
test_df = pd.DataFrame(test_gdf.drop(columns=['geometry','geometry_b']))

C:\Users\km20186\AppData\Local\Temp\ipykernel_32892\1463719098.py:1: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [15]:
# distance列がNULLとなっている割合を計算
null_ratio = train_gdf['distance'].isnull().sum() / len(train_gdf)
print(f"distance列のNULL割合: {null_ratio:.2%} ({train_gdf['distance'].isnull().sum()}/{len(train_gdf)})")


distance列のNULL割合: 12.05% (43843/363924)


In [52]:
# GeoDataFrameをDataFrameに変換(geometry列を削除)
train_gdf["land_price_x"]=train_gdf.geometry.x
train_gdf["land_price_y"]=train_gdf.geometry.y
train_df = pd.DataFrame(train_gdf.drop(columns=['geometry','geometry_b']))

In [53]:
train, test = train_test_split(train_df, test_size=0.2, random_state=42)
train_data: pd.DataFrame = TabularDataset(train)
test_data: pd.DataFrame = TabularDataset(test)
model=TabularPredictor(
    label='money_room',
    problem_type='regression',
    eval_metric='mape'
)
predictor=model.fit(
    train_data=train_data,
    test_data=test_data,
    presets='medium_quality',
    time_limit=360,
)

No path specified. Models will be saved in: "AutogluonModels\ag-20251210_103227"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       11.56 GB / 31.66 GB (36.5%)
Disk Space Avail:   75.38 GB / 237.73 GB (31.7%)
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       11.56 GB / 31.66 GB (36.5%)
Disk Space Avail:   75.38 GB / 237.73 GB (31.7%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit =

[1000]	valid_set's l2: 2.93693e+13	valid_set's mean_absolute_percentage_error: -0.169183
[2000]	valid_set's l2: 2.63227e+13	valid_set's mean_absolute_percentage_error: -0.161121
[2000]	valid_set's l2: 2.63227e+13	valid_set's mean_absolute_percentage_error: -0.161121
[3000]	valid_set's l2: 2.48318e+13	valid_set's mean_absolute_percentage_error: -0.156385
[3000]	valid_set's l2: 2.48318e+13	valid_set's mean_absolute_percentage_error: -0.156385
[4000]	valid_set's l2: 2.39373e+13	valid_set's mean_absolute_percentage_error: -0.153553
[4000]	valid_set's l2: 2.39373e+13	valid_set's mean_absolute_percentage_error: -0.153553
[5000]	valid_set's l2: 2.33337e+13	valid_set's mean_absolute_percentage_error: -0.151441
[5000]	valid_set's l2: 2.33337e+13	valid_set's mean_absolute_percentage_error: -0.151441


	Ran out of time, early stopping on iteration 5949. Best iteration is:
	[5945]	valid_set's l2: 2.29428e+13	valid_set's mean_absolute_percentage_error: -0.14987
	-0.1499	 = Validation score   (-mean_absolute_percentage_error)
	268.77s	 = Training   runtime
	0.43s	 = Validation runtime
	-0.1499	 = Validation score   (-mean_absolute_percentage_error)
	268.77s	 = Training   runtime
	0.43s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 268.03s of the -1.94s of remaining time.
Fitting model: WeightedEnsemble_L2 ... Training model for up to 268.03s of the -1.94s of remaining time.
	Ensemble Weights: {'LightGBMXT': 1.0}
	-0.1499	 = Validation score   (-mean_absolute_percentage_error)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
	Ensemble Weights: {'LightGBMXT': 1.0}
	-0.1499	 = Validation score   (-mean_absolute_percentage_error)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 362.85s ... B

In [39]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.15586,-0.153477,mean_absolute_percentage_error,11.307736,0.408052,271.648864,11.307736,0.408052,271.648864,1,True,1
1,WeightedEnsemble_L2,-0.15586,-0.153477,mean_absolute_percentage_error,11.368932,0.408052,271.666173,0.061197,0.000000,0.017309,2,True,2


In [54]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.151821,-0.14987,mean_absolute_percentage_error,8.931070,0.428252,268.771355,8.931070,0.428252,268.771355,1,True,1
1,WeightedEnsemble_L2,-0.151821,-0.14987,mean_absolute_percentage_error,8.985064,0.429255,268.780988,0.053993,0.001003,0.009634,2,True,2


In [47]:
test_data: pd.DataFrame = TabularDataset(test_df)
predictions = predictor.predict(test_data)
#

In [48]:
pred=pd.DataFrame(predictions)
pred.head()
pred.to_csv(os.path.join('..', 'data', 'submit', 'submission.csv'), index_label='id')

In [ ]:
train_data.head()

In [41]:
importance_df = predictor.feature_importance(test,num_shuffle_sets=1)

These features in provided data are not utilized by the predictor and will be ignored: ['building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing', 'L02_095']
Computing feature importance via permutation shuffling for 190 features using 5000 rows with 1 shuffle sets...
Computing feature importance via permutation shuffling for 190 features using 5000 rows with 1 shuffle sets...
	388.85s	= Expected runtime (388.85s per shuffle set)
	388.85s	= Expected runtime (388.85s per shuffle set)
	402.28s	= Actual runtime (Completed 1 of 1 shuffle sets)
	402.28s	= Actual runtime (Completed 1 of 1 shuffle sets)


In [44]:
display(importance_df.T)

,house_area,L02_006,year_built,rosen_name1,L02_094,snapshot_land_area,eki_name1,L02_092,madori_kind_all,walk_distance1,L02_093,unit_area,post1,rosen_name2,bukken_type,bus_time1,addr1_2,L02_001,building_structure,snapshot_create_date,room_floor,money_kyoueki,eki_name2,building_modify_date,school_ele_name,L02_045,L02_091,genkyo_code,building_land_area,school_jun_name,addr2_name,room_kaisuu,madori_number_all,floor_count,walk_distance2,building_create_date,L02_090,el,land_toshi,distance,reform_wet_area,statuses,land_price_x,floor_plan_code,lon,parking_money,land_youto,lat,unit_count,parking_kubun,parking_number,land_price_y,snapshot_modify_date,super_distance,bus_time2,new_date,L02_029,building_type,nl,unit_tag_id,room_count,L02_071,reform_wet_area_date,reform_interior,usable_date,L02_024,money_shuuzen,L02_089,building_land_chimoku,land_seigen,land_youseki,snapshot_land_shidou,money_kyoueki_tax,renovation_date,balcony_area,school_jun_distance,addr3_name,school_ele_distance,full_address,land_road_cond,dwelling_unit_window_angle,management_form,building_id,land_chisei,money_sonota_str1,post2,convenience_distance,land_kenpei,money_sonota1,building_name,L02_037,building_tag_id,money_rimawari_now,snapshot_window_angle,parking_money_tax,drugstore_distance,park_distance,timelimit_date,building_area_kind,homes_building_name,reform_interior_date,L02_082,L02_072,land_mochibun_b,land_area_all,land_area_kind,unit_area_max,L02_083,empty_number,house_kanrinin,land_setback_flg,empty_contents,L02_087,L02_060,L02_065,reform_exterior_date,land_setback,L02_062,unit_id,hospital_distance,reform_exterior,unit_name,L02_074,L02_068,parking_memo,L02_086,L02_070,L02_088,addr1_1,L02_084,target_ym,money_sonota_str2,L02_056,reform_interior_other,L02_055,est_other_distance,basement_floor_count,building_status,L02_085,L02_066,L02_058,parking_distance,building_area,flg_new,unit_area_min,L02_078,L02_069,renovation_etc,bus_stop1,L02_079,management_association_flg,L02_080,L02_059,L02_081,L02_073,est_other_name,parking_keiyaku,L02_075,land_shidou_a,L02_076,bank_distance,land_mochibun_a,money_sonota2,total_floor_area,L02_063,reform_wet_area_other,L02_067,homes_building_name_ruby,bukken_id,usable_status,reform_exterior_other,bus_stop2,L02_061,L02_057,reform_common_area,reform_common_area_date,money_sonota3,reform_place,land_shidou_b,traffic_car,reform_etc,L02_064,reform_date,traffic_other,shopping_street_distance,money_shuuzenkikin,reform_place_other,flg_investment,money_sonota_str3,L02_077
importance,0.256691,0.184866,0.179331,0.078714,0.05183,0.038195,0.024125,0.022314,0.019788,0.019471,0.019384,0.019373,0.017729,0.013745,0.013476,0.012119,0.011394,0.009005,0.006721,0.00501,0.004869,0.004735,0.004659,0.00456,0.004557,0.004475,0.004364,0.004333,0.004169,0.003772,0.003649,0.003605,0.003572,0.003062,0.002969,0.002968,0.002934,0.0029,0.002762,0.002579,0.00252,0.002406,0.002323,0.00232,0.002245,0.00222,0.002045,0.001999,0.001751,0.001713,0.001519,0.0015,0.001411,0.001304,0.001244,0.001138,0.001137,0.001115,0.001052,0.001047,0.001036,0.000982,0.000935,0.000902,0.000882,0.00087,0.00086,0.000859,0.000846,0.000845,0.000754,0.00073,0.00071,0.000709,0.000699,0.00069,0.000686,0.000669,0.000655,0.00065,0.000626,0.000621,0.000602,0.000597,0.000587,0.000578,0.000564,0.000535,0.000522,0.000512,0.00051,0.000497,0.000494,0.000487,0.000482,0.000462,0.000399,0.000397,0.000394,0.000382,0.000366,0.000366,0.000353,0.00034,0.00034,0.000333,0.000314,0.000296,0.000281,0.000278,0.000271,0.000259,0.000253,0.000247,0.00023,0.000229,0.000226,0.000212,0.000212,0.00021,0.000203,0.000203,0.00019,0.000178,0.000174,0.000169,0.000165,0.000164,0.000164,0.00016,0.000155,0.000152,0.000152,0.000152,0.00015,0.000146,0.000138,0.000136,0.000135,0.000127,0.000125,0.000122,0.000121,0.000116,0.000114,0.000113,0.00011,0.00011,0.000107,0.000103,0.0001,0.000097,0.000095,0.000088,0.000082,0.000079,0.000077,0.000075,0.000071,0.000066,0.000066,0.000065,0.000064,0.000058,0.000052,0.000051,0.000047,0.000046,0.000043,0.